In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob
from tqdm import tqdm
import pickle
from scipy import sparse
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from datetime import date, timedelta
import torch.optim as optim
import torch.nn as nn
from torch_geometric.nn import GCNConv
from torch_geometric import utils, data
from torch_geometric.loader import DataLoader
from sklearn.metrics import r2_score
pd.set_option('mode.chained_assignment',None)

In [52]:
zones = [int(z[3:]) for z in pd.read_csv('data/processed/SimpleNNData.csv', index_col=0).filter(regex = 'lz').columns]

In [91]:
def make_PTG(graph, zones):
    attr, adj = graph

    # Filter out 
    if (attr.time_to_reservation.values[-1] >= 48) or ~attr.next_customer[-1]:
        return None
    
    if attr.leave_zone[-1] not in zones:
        return None

    # Slice
    _, labels = sparse.csgraph.connected_components(csgraph=adj, directed=False, return_labels=True)
    newl = labels[-1]
    indices = labels == newl   

    attr = attr[indices]
    adj = adj[indices,:].tocsc()[:,indices].tocsr()

    # Time variables
    attr['weekend'] = attr.time.dt.weekday//5

    def circle_transform(col, max_val=86400):
        tot_sec = ((col - col.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
        cos_val = np.cos(2*np.pi*tot_sec/max_val)
        sin_val = np.sin(2*np.pi*tot_sec/max_val)
        return cos_val, sin_val

    attr['Time_Cos'], attr['Time_Sin'] = [x.values for x in circle_transform(attr.time)]

    # drop
    attr.drop(columns=['park_location_lat', 'park_location_long', 'leave_location_lat', 'leave_location_long', 'park_fuel', 'park_zone', 'moved', 'movedTF', 'time', 'prev_customer', 'next_customer'], inplace = True)

    # One hot encoding
    attr['leave_zone'] = pd.Categorical(attr['leave_zone'], categories=zones)
    attr = pd.get_dummies(attr, columns = ['leave_zone'], prefix='lz')

    attr['engine']= pd.Categorical(attr['engine'], categories=['118I', 'I3', 'COOPER', 'X1'])
    attr = pd.get_dummies(attr, columns = ['engine'], prefix='eng')

    # Get edges
    edge_index, edge_weight = utils.convert.from_scipy_sparse_matrix(adj)

    # Make pytorch data type
    d = data.Data(x = torch.tensor(attr.drop(columns = ['time_to_reservation']).to_numpy(dtype = 'float')).float(), edge_index=edge_index, edge_attr=edge_weight.float(), y = torch.tensor(attr.time_to_reservation.values).float())

    return d

In [169]:
files_raw = glob.glob("data/processed/Graphs/*")
files = [f for f in files_raw if '201908' in f]

dataset = []

for file in tqdm(files):
    with open(file, 'rb') as f:
        graph_collection = pickle.load(f)

    for g in graph_collection.values():
        res = make_PTG(g,zones)
        if res:
            dataset.append(res)

100%|██████████| 17/17 [13:14<00:00, 46.74s/it]


In [50]:
sdate = date(2019, 8, 15)   # start date
edate = date(2019, 12, 18)   # end date
delta = edate - sdate       # as timedelta
files = ['graphs/'+(sdate + timedelta(days=i)).strftime("%Y%m%d")+'.pickle' for i in range(delta.days + 1)]

['graphs/20190815.pickle',
 'graphs/20190816.pickle',
 'graphs/20190817.pickle',
 'graphs/20190818.pickle',
 'graphs/20190819.pickle',
 'graphs/20190820.pickle',
 'graphs/20190821.pickle',
 'graphs/20190822.pickle',
 'graphs/20190823.pickle',
 'graphs/20190824.pickle',
 'graphs/20190825.pickle',
 'graphs/20190826.pickle',
 'graphs/20190827.pickle',
 'graphs/20190828.pickle',
 'graphs/20190829.pickle',
 'graphs/20190830.pickle',
 'graphs/20190831.pickle',
 'graphs/20190901.pickle',
 'graphs/20190902.pickle',
 'graphs/20190903.pickle',
 'graphs/20190904.pickle',
 'graphs/20190905.pickle',
 'graphs/20190906.pickle',
 'graphs/20190907.pickle',
 'graphs/20190908.pickle',
 'graphs/20190909.pickle',
 'graphs/20190910.pickle',
 'graphs/20190911.pickle',
 'graphs/20190912.pickle',
 'graphs/20190913.pickle',
 'graphs/20190914.pickle',
 'graphs/20190915.pickle',
 'graphs/20190916.pickle',
 'graphs/20190917.pickle',
 'graphs/20190918.pickle',
 'graphs/20190919.pickle',
 'graphs/20190920.pickle',
 

In [2]:
with open('tmpdataset', 'rb') as f:
        ds = pickle.load(f)

In [36]:
train_val_size = int(0.8 * len(ds))
val_test_size = len(ds)-train_val_size
train_val_data, test_data = torch.utils.data.random_split(ds, [train_val_size, val_test_size])
train_size = train_val_size-val_test_size
train_data, val_data = torch.utils.data.random_split(train_val_data, [train_size, val_test_size])

with open('tmpdataset', 'wb') as handle:
    pickle.dump(dataset, handle, pickle.HIGHEST_PROTOCOL)

In [172]:
with open('tmpdataset', 'wb') as handle:
    pickle.dump(dataset, handle, pickle.HIGHEST_PROTOCOL)

In [157]:
from torch_geometric.loader import DataLoader
dataset = []
i = 0
for g in tqdm(graph_collection_t.values()):
    res = make_PTG(g,zones)
    if res:
        dataset.append(res)

train_loader = DataLoader(dataset, batch_size=128, shuffle=True)

100%|██████████| 2874/2874 [00:34<00:00, 84.22it/s]


In [158]:
from torch_geometric.loader import DataLoader
dataset = []
i = 0
for g in tqdm(graph_collection_v.values()):
    res = make_PTG(g,zones)
    if res:
        dataset.append(res)

val_loader = DataLoader(dataset, batch_size=128, shuffle=True)

100%|██████████| 2988/2988 [00:38<00:00, 77.04it/s]


In [160]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(269, 64)
        self.conv2 = GCNConv(64, 64)
        self.conv3 = GCNConv(64, 1)

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr

        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        #x = F.dropout(x, p = 0.2, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_weight)

        return x.squeeze()

GNN = GCN()
print(GNN, sum(p.numel() for p in GNN.parameters()))

GCN(
  (conv1): GCNConv(269, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 1)
) 21505


In [161]:
optimizer = optim.Adam(GNN.parameters(), lr=0.03, weight_decay = 0.0001) #Chaged to Adam and learning + regulariztion rate set
criterion = nn.MSELoss(reduction = 'mean')

In [162]:
# Set number of epochs
num_epochs = 10

# Set up lists for loss/R2
train_r2, train_loss = [], []
valid_r2, valid_loss = [], []
cur_loss = 0
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    ### Train
    cur_loss_train = []
    GNN.train()
    for batch in train_loader:
        optimizer.zero_grad()
        out = GNN(batch)
        batch_loss = criterion(out[batch.ptr[1:]-1],batch.y[batch.ptr[1:]-1])
        batch_loss.backward()
        optimizer.step()

        cur_loss_train.append(batch_loss.item())
    
    train_losses.append(np.mean(cur_loss_train))

    ### Evaluate training
    GNN.eval()
    train_preds, train_targs = [], []
    for batch in train_loader:
        preds = GNN(batch)
        train_targs += list(batch.y.numpy()[batch.ptr[1:]-1])
        train_preds += list(preds.detach().numpy()[batch.ptr[1:]-1])


    ### Evaluate validation
    val_preds, val_targs = [], []
    cur_loss_val = []
    for batch in val_loader:
        preds = GNN(batch)[batch.ptr[1:]-1]
        y_val = batch.y[batch.ptr[1:]-1]
        val_targs += list(y_val.numpy())
        val_preds += list(preds.detach().numpy())
        cur_loss_val.append(criterion(preds, y_val).item())

    val_losses.append(np.mean(cur_loss_val))


    train_r2_cur = r2_score(train_targs, train_preds)
    valid_r2_cur = r2_score(val_targs, val_preds)
    
    train_r2.append(train_r2_cur)
    valid_r2.append(valid_r2_cur)

    print("Epoch %2i: Train Loss %f , Valid Loss %f ,Train R2 %f, Valid R2 %f" % (
                epoch+1, train_losses[-1], val_losses[-1],train_r2_cur, valid_r2_cur))

Epoch  1: Train Loss 66.624082 , Valid Loss 15.706058 ,Train R2 0.013291, Valid R2 0.027282
Epoch  2: Train Loss 16.410148 , Valid Loss 15.844169 ,Train R2 0.054510, Valid R2 0.020674
Epoch  3: Train Loss 15.411344 , Valid Loss 14.727145 ,Train R2 0.159547, Valid R2 0.098666
Epoch  4: Train Loss 15.844001 , Valid Loss 14.511927 ,Train R2 0.180841, Valid R2 0.118863
Epoch  5: Train Loss 14.283771 , Valid Loss 14.749466 ,Train R2 0.197472, Valid R2 0.084591
Epoch  6: Train Loss 13.901943 , Valid Loss 14.692825 ,Train R2 0.229472, Valid R2 0.099606
Epoch  7: Train Loss 13.852442 , Valid Loss 14.578636 ,Train R2 0.236093, Valid R2 0.092907
Epoch  8: Train Loss 13.923434 , Valid Loss 15.180002 ,Train R2 0.203826, Valid R2 0.067741
Epoch  9: Train Loss 14.383020 , Valid Loss 15.570014 ,Train R2 0.217375, Valid R2 0.044942
Epoch 10: Train Loss 14.358528 , Valid Loss 15.396537 ,Train R2 0.209249, Valid R2 0.045433


In [85]:
# setting hyperparameters and gettings epoch sizes
batch_size = 4084
num_epochs = 121
num_samples_train = X_train.shape[0]
num_batches_train = num_samples_train // batch_size
num_samples_valid = X_val.shape[0]
num_batches_valid = num_samples_valid // batch_size

# setting up lists for handling loss/accuracy
train_r2, train_loss = [], []
valid_r2, valid_loss = [], []
test_acc, test_loss = [], []
cur_loss = 0
train_losses = []
val_losses = []

get_slice = lambda i, size: range(i * size, (i + 1) * size)

early_stopping = EarlyStopping(patience=10, verbose=False)

for epoch in range(num_epochs):
    # Forward -> Backprob -> Update params
    ## Train
    cur_loss_train = []
    net.train()
    for i in range(num_batches_train):
        optimizer.zero_grad()
        slce = get_slice(i, batch_size)
        output = net(X_train[slce])
        
        # compute gradients given loss
        target_batch = y_train[slce]
        batch_loss = criterion(output, target_batch)
        batch_loss.backward()
        optimizer.step()
        
        cur_loss_train.append(batch_loss.item())
    train_losses.append(np.mean(cur_loss_train))

    net.eval()
    ### Evaluate training
    train_preds, train_targs = [], []
    for i in range(num_batches_train):
        slce = get_slice(i, batch_size)
        output = net(X_train[slce])
        
        preds = output
        
        train_targs += list(y_train[slce].numpy())
        train_preds += list(preds.data.numpy())
    
    ### Evaluate validation
    val_preds, val_targs = [], []
    cur_loss_val = []
    for i in range(num_batches_valid):
        slce = get_slice(i, batch_size)
        
        output = net(X_val[slce])
        preds = output
        val_targs += list(y_val[slce].numpy())
        val_preds += list(preds.data.numpy())

        cur_loss_val.append(criterion(output, y_val[slce]).item())

    val_losses.append(np.mean(cur_loss_val))


    train_r2_cur = r2_score(train_targs, train_preds)
    valid_r2_cur = r2_score(val_targs, val_preds)
    
    train_r2.append(train_r2_cur)
    valid_r2.append(valid_r2_cur)

    # EarlyStopping
    early_stopping(val_losses[-1], net)
    if early_stopping.early_stop:
        print("Early stopping")
        print("Epoch %2i: Train Loss %f , Valid Loss %f , Train R2 %f, Valid R2 %f" % (
            epoch+1, train_losses[-1], val_losses[-1], train_r2_cur, valid_r2_cur))
        break
    
    if epoch % 10 == 0:
        print("Epoch %2i: Train Loss %f , Valid Loss %f ,Train R2 %f, Valid R2 %f" % (
                epoch+1, train_losses[-1], val_losses[-1],train_r2_cur, valid_r2_cur))

epoch = np.arange(len(train_r2))
plt.figure()
plt.plot(epoch, train_r2, 'r', epoch, valid_r2, 'b')
plt.legend(['Train Accucary','Validation Accuracy'])
plt.xlabel('Updates'), plt.ylabel('R2')

NameError: name 'X_train' is not defined